In [5]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
import tensorflow as tf
import numpy as np

# load mnist data
mnist = input_data.read_data_sets("../../DeepLearning/TensorFlow/MNIST_data/", one_hot=True)

# define parameter 
learning_rate = 0.001
training_iters = 50000
batch_size = 128
display_step = 10
dropout = 0.75

n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784 # 28x28
n_classes =10 # output

Extracting ../../DeepLearning/TensorFlow/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../DeepLearning/TensorFlow/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../DeepLearning/TensorFlow/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../DeepLearning/TensorFlow/MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
# define placeholder
# shape of [-1] flattens into 1-D
x = tf.placeholder("float", [None, n_input])
# construct data
_X = tf.reshape(x, shape=[-1, 28, 28, 1]) 
y = tf.placeholder("float", [None, n_classes])

# first convolution layer - 32 features with 5x5 tensor
wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
bc1 = tf.Variable(tf.random_normal([32]))
# Must have strides[0] = strides[3] = 1. 
# For the most common case of the same horizontal and vertices strides, 
# strides = [1, stride, stride, 1].
# padding = "SAME" 是指說input layer與第一層的卷積結果是相同維度的，ex:input=28x28 conv1=28x28xfeatures
conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_X, filter=wc1, strides=[1, 1, 1, 1], padding='SAME'), bc1))

# max pooling
k = 2
conv1 = tf.nn.max_pool(conv1, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# dropout
keep_prob = tf.placeholder(tf.float32)
conv1 = tf.nn.dropout(conv1, keep_prob)

# second convolution layer
wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
bc2 = tf.Variable(tf.random_normal([64]))
conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv1, filter=wc2, strides=[1, 1, 1, 1], padding='SAME'), bc2))
k = 2
conv2 = tf.nn.max_pool(conv2, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
conv2 = tf.nn.dropout(conv2, keep_prob)

# flatten 
wd1 = tf.Variable(tf.random_normal([7*7*64, 1024]))
bd1 = tf.Variable(tf.random_normal([1024]))
dense1 = tf.reshape(conv2, [-1, wd1.get_shape().as_list()[0]])
dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, wd1), bd1))
dense1 = tf.nn.dropout(dense1, keep_prob)

# fully connection layer
wout = tf.Variable(tf.random_normal([1024, n_classes]))
bout = tf.Variable(tf.random_normal([n_classes]))
pred = tf.add(tf.matmul(dense1, wout), bout)

# cost function & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# run
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys, keep_prob: dropout})
        if step % display_step ==0:
            acc = sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys, keep_prob: 1.})
            loss = sess.run(cost, feed_dict={x:batch_xs, y:batch_ys, keep_prob: 1.})
            print("Iter:{}, Minibatch Loss:{}, training accuracy:{}".format(str(step*batch_size),
                                                                           loss, acc))
        step += 1
    print("Test accuracy:{}".format(sess.run(accuracy, feed_dict={x:mnist.test.images[:256], 
                                                                  y:mnist.test.labels[:256], 
                                                                  keep_prob: 1.})))
print("Finished!")

Iter:1280, Minibatch Loss:49660.33203125, training accuracy:0.09375
Iter:2560, Minibatch Loss:31652.541015625, training accuracy:0.25
Iter:3840, Minibatch Loss:17414.33984375, training accuracy:0.3359375
Iter:5120, Minibatch Loss:13953.4794921875, training accuracy:0.3984375
Iter:6400, Minibatch Loss:9298.1953125, training accuracy:0.59375
Iter:7680, Minibatch Loss:9277.0205078125, training accuracy:0.53125
Iter:8960, Minibatch Loss:8109.361328125, training accuracy:0.5234375
Iter:10240, Minibatch Loss:6327.27587890625, training accuracy:0.65625
Iter:11520, Minibatch Loss:3052.865234375, training accuracy:0.75
Iter:12800, Minibatch Loss:6686.3974609375, training accuracy:0.6875
Iter:14080, Minibatch Loss:2769.619384765625, training accuracy:0.7109375
Iter:15360, Minibatch Loss:3250.087890625, training accuracy:0.765625
Iter:16640, Minibatch Loss:4017.419921875, training accuracy:0.78125
Iter:17920, Minibatch Loss:1828.7677001953125, training accuracy:0.8515625
Iter:19200, Minibatch Los